# Final Analysis_2 

## 1. Problem statement (summarized)

- How are the negative reviews distribution look like, given we have already identify groups of sellers to focus on.


Our key findings so far :
- The `wait_time` was the most significant factor behind low review scores, but reading comments of the bad reviews also showed that some of them were linked to the seller or to the product itself.
- `wait_time` is made up of seller's `delay_to_carrier` + `carrier_delivery_time`. The latter being outside of Olist's direct control, improving it is not a quick-win recommendation we can make to Olist CEO without in-depth analysis of their operational practices.
- On the contrary, a better selection of `sellers` can positively impact the `delay_to_carrier` and reduce the number of bad `review_scores` on Olist.

In [1]:
from olist.data import Olist

import translators as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Using state Perak server backend.


In [34]:
data = Olist().get_data()
reviews = data["order_reviews"]

one_star_reviews = reviews[(reviews["review_score"] == 1) & (~reviews["review_comment_message"].isna())]
one_star_reviews.loc[:,'review_comment_message'] = one_star_reviews['review_comment_message'].astype('str')
print(one_star_reviews.shape)
one_star_reviews.head()

(8745, 7)


/home/ysin/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
19,373cbeecea8286a2b66c97b1b157ec46,583174fbe37d3d5f0d6661be3aad1786,1,Não chegou meu produto,Péssimo,2018-08-15 00:00:00,2018-08-15 04:10:37
29,2c5e27fc178bde7ac173c9c62c31b070,0ce9a24111d850192a933fcaab6fbad3,1,NaN,Não gostei ! Comprei gato por lebre,2017-12-13 00:00:00,2017-12-16 07:14:07
32,58044bca115705a48fe0e00a21390c54,68e55ca79d04a79f20d4bfc0146f4b66,1,NaN,Sempre compro pela Internet e a entrega ocorre...,2018-04-08 00:00:00,2018-04-09 12:22:39
39,9fd59cd04b42f600df9f25e54082a8d1,3c314f50bc654f3c4e317b055681dff9,1,NaN,Nada de chegar o meu pedido.,2017-04-21 00:00:00,2017-04-23 05:37:03
51,e233e51d11511bf30e568c76360ace52,548df2c6e5f089574614894bca78acf5,1,NaN,recebi somente 1 controle Midea Split ESTILO.\...,2018-01-28 00:00:00,2018-02-04 11:06:15


In [35]:
def translation(data,series):
    name = "translated_"+series.name.split("_")[-1]
    
    for i in range(series.shape[0]):
        data.loc[i,name]= ts.google(data.loc[i,series.name], from_language = "pt", to_language = "en")

In [25]:
%%time
resampled_reviews = one_star_reviews.sample(50).reset_index()
translation(resampled_reviews, resampled_reviews['review_comment_message'])
resampled_reviews

CPU times: user 1.14 s, sys: 80.6 ms, total: 1.22 s
Wall time: 1min 24s


,index,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,translated_message
0,50133,2521ded73db8561e0fd8df85eeeaaf5e,265271dd979e1221bab55d039af24b37,1,nan,O vendedor entrou em contato comigo mas não co...,2018-01-12 00:00:00,2018-01-12 10:53:59,The seller contacted me but did not conclude t...
1,90102,afbdacbc3c4264261766988251038ef3,72e4ee7d458b36222813e2b9707d9d25,1,nan,comprei 4 produtos na americana a quase 1 mês ...,2017-12-21 00:00:00,2017-12-21 22:25:54,I bought 4 products in the Americana almost 1 ...
2,11047,f495b89455116ac326d54a32ee939a00,2dee1b58f6f3c724427ad545adf6d5e4,1,nan,Boa noite \r\nO produto não foi entregue e ain...,2018-04-07 00:00:00,2018-04-07 23:05:19,Goodnight \r\n The product has not been delive...
3,38740,f763252b810a719d655523cd5b748c37,e43508515d5e7a615643e552dc54a970,1,nan,nao tenho como avaliar ja que ainda nao recebi,2018-04-08 00:00:00,2018-04-09 00:48:05,I can't evaluate since I haven't received it yet
4,42099,2af2f4bb459f8df919aaaadbe64e7cad,a9e3336634f202f7806561c5f41b2e1a,1,Indignada,Comprei começo do mês passado.Passou quase 1 m...,2018-05-10 00:00:00,2018-05-10 05:19:48,I bought the beginning of last month. I set al...
5,66728,a765ade5a94437b1d9dd9969f30149f2,e47c4c77d188f893a6534dbca93d1ef0,1,nan,Não recebi meu produto e não recebi meu dinhei...,2017-06-15 00:00:00,2017-06-16 21:20:18,I did not receive my product and did not recei...
6,11896,d0517d82a887cfa4d60bbd1790ec563b,5f3ce710839dcf62389d5187ece2d4e8,1,Me enganaram,"Fiz a compra de 2 conjuntos, e apesar de const...",2018-07-29 00:00:00,2018-07-30 14:29:37,"I purchased 2 sets, and although it appears in..."
7,94004,b7013a9e63f944b85e8e44ee177f0b0c,6adb948b069c269ba13ff1c38ad90228,1,nan,mandaram meu pedido errado ja mandei o email e...,2017-05-31 00:00:00,2017-06-01 09:30:03,sent my wrong order already sent the email and...
8,37848,55bc097bf380cffd36a1d0a9efe200ff,e3db241d7377ac91f1253454bdbeafe4,1,nan,comprei dia 12 e ate hj dia 28 nao recebi e os...,2017-06-28 00:00:00,2017-06-28 19:38:31,I bought on the 12th and until today the 28th ...
9,27107,dd3e09a802675667fb9761109edea549,e95f655fc4bb105e8915ab28d7bd0ec6,1,Nome incompleto,Não recebi o produto pois a loja não colocou m...,2018-05-12 00:00:00,2018-05-14 21:54:56,I did not receive the product because the stor...


Review title is not used as most of them are empty. As seen above, we sampled 50 reviews and translated their comment message using google translator package. Translation took roughly 1.3 minutes for 50 reviews. Due to the time constraint (all reviews - 8745 of them will need around 4 hours to translate), we will only sample around 500 reviews to for analysis.